# December 16, 2022
https://adventofcode.com/2022/day/16

In [1]:
import re
import numpy as np

In [2]:
class Valve:
    num_valves = 0

    def __init__( self, name, flow, neighbors ):
        self.name = name
        self.flow = flow
        self.status = "on" if self.flow == 0 else "off"
        self.neighbors = {nbr:1 for nbr in neighbors}
        self.id = Valve.num_valves
        Valve.num_valves += 1

    def is_on(self):
        return True if self.status == "on" else False

    def turn_on(self):
        if self.is_on:
            return 0
        else:
            self.status = "on"
            return self.flow

    def __str__(self):
        return f'''Valve #{self.id} {self.name} is {self.status}. Flow = {self.flow}. Neighbors: {self.neighbors}'''

    def __repr__(self):
        return str(self)

In [3]:
def parse_map( fn ):
    valves = []

    with open(fn, "r") as file:
        while True:
            line = file.readline().strip("\n")
            if not line: break
            ma = re.match(r'Valve (\w+).*rate=(\d+).*valves? ([\w,\s]*)', line)
            name = ma.group(1)
            flow = int(ma.group(2))
            nbrs = ma.group(3).split(", ")
            valves.append( Valve(name, flow, nbrs) )

    return valves


In [4]:
test = parse_map("data/16_test.txt")

In [5]:
print("\n".join([str(x) for x in test]))

Valve #0 AA is on. Flow = 0. Neighbors: {'DD': 1, 'II': 1, 'BB': 1}
Valve #1 BB is off. Flow = 13. Neighbors: {'CC': 1, 'AA': 1}
Valve #2 CC is off. Flow = 2. Neighbors: {'DD': 1, 'BB': 1}
Valve #3 DD is off. Flow = 20. Neighbors: {'CC': 1, 'AA': 1, 'EE': 1}
Valve #4 EE is off. Flow = 3. Neighbors: {'FF': 1, 'DD': 1}
Valve #5 FF is on. Flow = 0. Neighbors: {'EE': 1, 'GG': 1}
Valve #6 GG is on. Flow = 0. Neighbors: {'FF': 1, 'HH': 1}
Valve #7 HH is off. Flow = 22. Neighbors: {'GG': 1}
Valve #8 II is on. Flow = 0. Neighbors: {'AA': 1, 'JJ': 1}
Valve #9 JJ is off. Flow = 21. Neighbors: {'II': 1}


In [6]:
def create_graph( valves ):
    N = len(valves)
    name_map = {v.name:i for i,v in enumerate(valves)}
    dist_array = np.array( [-1]*N*N ).reshape( [N,N])
    for i in range(N):
        dist_array[i,i] = 0

    for cur_id, cur in enumerate(valves):
        dist = 0
        while True:
            # list of valves not reached yet
            tofind = [ i for i,d in enumerate( dist_array[cur_id,:] ) if d == -1 ]
            
            # end state is only the diagonals are 0
            if len(tofind) == 0:
                break
            
            # find most recent neighbors
            cur_nbrs = [ i for i,d in enumerate( dist_array[cur_id,] ) if d == dist ]
            if len(cur_nbrs) == 0:
                # uhoh--- can't get there from here!
                break

            # find neighbors of neighbors that aren't already in the distance array
            next_nbrs = set()
            for d_nbr in cur_nbrs:
                d_nbr_nbr = [ name_map[nbr] for nbr in valves[d_nbr].neighbors if name_map[nbr] in tofind ] 
                next_nbrs = next_nbrs.union( d_nbr_nbr )

            # add the new neighbor to the distance arracy
            dist += 1
            for id in next_nbrs:
                dist_array[cur_id, id] = dist
                dist_array[id, cur_id] = dist

            # end while loop for finding out cur_id distances
    return dist_array

In [7]:
test_graph = create_graph(test)
test_graph

array([[0, 1, 2, 1, 2, 3, 4, 5, 1, 2],
       [1, 0, 1, 2, 3, 4, 5, 6, 2, 3],
       [2, 1, 0, 1, 2, 3, 4, 5, 3, 4],
       [1, 2, 1, 0, 1, 2, 3, 4, 2, 3],
       [2, 3, 2, 1, 0, 1, 2, 3, 3, 4],
       [3, 4, 3, 2, 1, 0, 1, 2, 4, 5],
       [4, 5, 4, 3, 2, 1, 0, 1, 5, 6],
       [5, 6, 5, 4, 3, 2, 1, 0, 6, 7],
       [1, 2, 3, 2, 3, 4, 5, 6, 0, 1],
       [2, 3, 4, 3, 4, 5, 6, 7, 1, 0]])

### Part 1

Functions for memoization

In [9]:
def get_key( start, time_left, status ):
    return f'''{start}.{time_left}.{"".join([str(s) for s in status])}'''

def memoize( score, start, time_left, status, best=None ):
    key = get_key(start, time_left, status)
    if best is None:
        best = {key:score}
    else:
        best[key] = score
    return best

def read_memo( start, time_left, status, best=None ):
    if best is None:
        return None
    key = get_key(start, time_left, status)
    if key in best:
        return best[key]
    else:
        return None

In [10]:
def solve_map( valves, graph, start, time_left, status=None, best=None ):
    # convert to status vector to a string so we don't need to copy the whole Valve object
    if status is None:
        status = [ 1 if v.status == "on" else 0 for v in valves ]

    # read memo if possible
    best_result = read_memo( start, time_left, status, best )
    if best_result is not None:
        return best_result

    # Not previously solved. Let's solve it now!
    best_result = 0


    if time_left > 0:
        # possible places we can make it to
        # -- valve is off (s == 0)
        # -- valve is reachable and not current position (dist > 0)
        # -- valve is reachable in time (diat < time_left. We don't use <= since there must be time for an action there.)
        poss = [i for i,s in enumerate(status) if s == 0 and graph[start,i] > 0 and graph[start,i] < time_left]

        # Move to loc and turn on that valve
        for loc in poss:
            new_status = status.copy()
            new_status[loc] = 1
            new_time_left = time_left - graph[start, loc] - 1 # +1 for turning off that valve
            result = valves[loc].flow * new_time_left
            result += solve_map( valves, graph, loc, new_time_left, new_status, best )
            best_result = max(best_result, result)

    best = memoize( best_result, start, time_left, status, best=None )
    return best_result    

In [11]:
solve_map( test, test_graph, 0, 30 )

1651

In [12]:
# 16.7s without memoization
# 8.1 with memoization
puz = parse_map("data/16.txt")
puz_graph = create_graph(puz)
start = [i for i,v in enumerate(puz) if v.name=="AA"][0]
print(f'''Starting at Valve #{start} ({puz[start].name}).''')
solve_map( puz, puz_graph, start, 30 )


Starting at Valve #14 (AA).


2320

### Part 2

In [14]:
def get_key( start1, start2, time_left1, time_left2, status ):
    return f'''{start1}.{start2}.{time_left1}.{time_left2}.{"".join([str(s) for s in status])}'''

def memoize( score, start1, start2, time_left1, time_left2, status, best=None ): 
    key = get_key( start1, start2, time_left1, time_left2, status)

    if best is None:
        best = {key: score}
    else:
        best[key] = score
    return best

def read_memo( start1, start2, time_left1, time_left2, status, best=None ):
    key = get_key( start1, start2, time_left1, time_left2, status)

    if best is not None and key in best:
        best[key]
    else:
        return None

def get_status_cd( status ):
    return sum([ s*(10**i) for i,s in enumerate(status) ])

def get_time_left_cd( tl1, tl2 ):
    return tl1*100 + tl2

def get_start_cd(start1, start2):
    return start1*100 + start2

In [17]:
def solve_map( valves, graph, start1, start2, time_left1, time_left2, status=None, best=None ):
    # convert to status vector to a string so we don't need to copy the whole Valve object
    if status is None:
        status = [ 1 if v.status == "on" else 0 for v in valves ]

    if time_left2 > time_left1:
        # simplify code by making id1 the person with more time left
        tmp = start2
        start2 = start1
        start1 = tmp

        tmp = time_left2
        time_left2 = time_left1
        time_left1 = tmp

    # read memo if possible
    best_result = read_memo( start1, start2, time_left1, time_left2, status, best )
    if best_result is not None:
        return best_result
    
    # Not previously solved. Let's solve it now!
    best_result = 0


    if time_left1 > 0 or time_left2 > 0:
        # possible places we can make it to
        # -- valve is off (s == 0)
        # -- valve is reachable and not current position (dist > 0)
        # -- valve is reachable in time (dist < time_left1. We don't use <= since there must be time for an action there.)
        poss = [i for i,s in enumerate(status) if s == 0 and graph[start1,i] > 0 and graph[start1,i] < time_left1]

        # edge case: person with less time is only one who can turn on an extra valve (ie. it's closer to one)
        if len(poss) == 0:
            result = solve_map(valves, graph, start1, start2, 0, time_left2, status, best)
            best_result = max(best_result, result)
        
        else:
            # Move to loc and turn on that valve
            for loc in poss:
                new_status = status.copy()
                new_status[loc] = 1
                new_time_left1 = time_left1 - graph[start1, loc] - 1 # +1 for turning off that valve
                result = valves[loc].flow * new_time_left1
                result += solve_map( valves, graph, loc, start2, new_time_left1, time_left2, new_status, best )
                best_result = max(best_result, result)
    else:
        global end_count
        end_count += 1
        if end_count % 100000 == 0:
            print(f'''Followed {end_count} chains to the end''')

    # could improve by not memoizing 0s
    best = memoize( best_result, start1, start2, time_left1, time_left2, status, best )

    return best_result    

In [19]:
end_count = 0
solve_map( test, test_graph, 0, 0, 26, 26 )


1707

In [29]:
import sys
puz = parse_map("data/16.txt")
puz_graph = create_graph(puz)
start = [i for i,v in enumerate(puz) if v.name=="AA"][0]
print(f'''Starting at Valve #{start} ({puz[start].name}).''')

is_off = [i for i,v in enumerate(puz) if v.status=="off"]
Noff = len(is_off)
puz_off = np.zeros(Noff*Noff).reshape([Noff, Noff])
for i in range(Noff):
    for j in range(Noff):
        puz_off[i,j] = puz_graph[is_off[i], is_off[j] ]

# Looking for some meaningful structure to help eliminate paths
# Nothing super obvious. Maybe the middle valves tend to be more centrally located
np.set_printoptions(threshold=sys.maxsize)
print(puz_off)

Starting at Valve #14 (AA).
[[ 0. 12.  2.  9.  5. 12.  5.  5. 10. 12. 14.  2.  5.  2.  7.]
 [12.  0. 10.  7.  8.  2.  7.  7.  2.  4.  2. 11. 13. 10.  6.]
 [ 2. 10.  0.  7.  5. 10.  3.  5.  8. 10. 12.  3.  3.  3.  5.]
 [ 9.  7.  7.  0.  4.  5.  4.  5.  5.  3.  9.  7. 10.  8.  2.]
 [ 5.  8.  5.  4.  0.  8.  2.  3.  6.  7. 10.  3.  8.  6.  2.]
 [12.  2. 10.  5.  8.  0.  7.  7.  2.  2.  4. 11. 13. 10.  6.]
 [ 5.  7.  3.  4.  2.  7.  0.  2.  5.  7.  9.  5.  6.  5.  2.]
 [ 5.  7.  5.  5.  3.  7.  2.  0.  5.  8.  9.  6.  8.  3.  3.]
 [10.  2.  8.  5.  6.  2.  5.  5.  0.  3.  4.  9. 11.  8.  4.]
 [12.  4. 10.  3.  7.  2.  7.  8.  3.  0.  6. 10. 13. 11.  5.]
 [14.  2. 12.  9. 10.  4.  9.  9.  4.  6.  0. 13. 15. 12.  8.]
 [ 2. 11.  3.  7.  3. 11.  5.  6.  9. 10. 13.  0.  6.  3.  5.]
 [ 5. 13.  3. 10.  8. 13.  6.  8. 11. 13. 15.  6.  0.  6.  8.]
 [ 2. 10.  3.  8.  6. 10.  5.  3.  8. 11. 12.  3.  6.  0.  6.]
 [ 7.  6.  5.  2.  2.  6.  2.  3.  4.  5.  8.  5.  8.  6.  0.]]


In [38]:
# 61m --- 11714002 end points (3 with tl==26)
# + 624m --- 
end_count = 0
#best = {}
solve_map( puz, puz_graph, start, start, 26, 26, best=best )

Followed 100000 chains to the end
Followed 200000 chains to the end
Followed 300000 chains to the end
Followed 400000 chains to the end
Followed 500000 chains to the end
Followed 600000 chains to the end
Followed 700000 chains to the end
Followed 800000 chains to the end
Followed 900000 chains to the end
Followed 1000000 chains to the end
Followed 1100000 chains to the end
Followed 1200000 chains to the end
Followed 1300000 chains to the end
Followed 1400000 chains to the end
Followed 1500000 chains to the end
Followed 1600000 chains to the end
Followed 1700000 chains to the end
Followed 1800000 chains to the end
Followed 1900000 chains to the end
Followed 2000000 chains to the end
Followed 2100000 chains to the end
Followed 2200000 chains to the end
Followed 2300000 chains to the end
Followed 2400000 chains to the end
Followed 2500000 chains to the end
Followed 2600000 chains to the end
Followed 2700000 chains to the end
Followed 2800000 chains to the end
Followed 2900000 chains to th

KeyboardInterrupt: 

In [39]:
len(best.keys())

18707331

In [40]:
tl = [int(k.split(".")[2]) for k in best.keys()]

In [41]:
sum([1 for x in tl if x ==26])

7

In [32]:
best

{'13.25.0.0.11111111111111111111111111111111111000111111110100011111011': 0,
 '13.25.1.0.11111111111111111111111111111111111000111111110100011111011': 0,
 '25.13.1.1.11111111111111111111111111111111111000111111110100011111011': 0,
 '13.35.0.0.11111111111111111111111110111111111100111111110100011111011': 0,
 '13.35.1.0.11111111111111111111111110111111111100111111110100011111011': 0,
 '13.48.0.0.11111111111111111111111110111111111000111111110110011111011': 0,
 '13.48.1.0.11111111111111111111111110111111111000111111110110011111011': 0,
 '13.56.0.0.11111111111111111111111110111111111000111111110100011111111': 0,
 '13.56.1.0.11111111111111111111111110111111111000111111110100011111111': 0,
 '56.13.1.1.11111111111111111111111110111111111000111111110100011111111': 0,
 '9.13.4.1.11111111111111111111111110111111111000111111110100011111011': 5,
 '25.35.0.0.11111111111110111111111111111111111100111111110100011111011': 0,
 '25.35.2.0.11111111111110111111111111111111111100111111110100011111011': 0,
